In [1]:
import numpy as np
import sys
import time
import h5py
import tensorflow as tf
from tqdm import tqdm

import numpy as np
import re
from math import ceil
from sklearn.metrics import average_precision_score

import keras.backend as kb
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Activation
from keras.layers.convolutional import Conv1D, Cropping1D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import add
from keras.layers import Input
from keras.layers.merge import concatenate
from keras.layers.core import Lambda

import pandas as pd
import pickle
from scipy.sparse import load_npz
from glob import glob
from tensorflow.keras.models import load_model 
import torch

from src.evaluation_metrics import print_topl_statistics,cross_entropy_2d
from src.dataloader import getData,spliceDataset,h5pyDataset,getDataPointListFull,DataPoint

2022-10-10 11:22:32.495646: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Using TensorFlow backend.


In [2]:
!nvidia-smi

Mon Oct 10 11:22:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.43.04    Driver Version: 515.43.04    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:31:00.0 Off |                    0 |
| N/A   38C    P0    38W / 250W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  Off  | 00000000:98:00.0 Off |                    0 |
| N/A   

In [3]:

###############################################################################
# Model
###############################################################################

L = 32
N_GPUS = 2

#if int(sys.argv[1]) == 80:
#    W = np.asarray([11, 11, 11, 11])
#    AR = np.asarray([1, 1, 1, 1])
#    BATCH_SIZE = 18*N_GPUS
#elif int(sys.argv[1]) == 400:
#    W = np.asarray([11, 11, 11, 11, 11, 11, 11, 11])
#    AR = np.asarray([1, 1, 1, 1, 4, 4, 4, 4])
#    BATCH_SIZE = 18*N_GPUS
#elif int(sys.argv[1]) == 2000:
#    W = np.asarray([11, 11, 11, 11, 11, 11, 11, 11,
#                    21, 21, 21, 21])
#    AR = np.asarray([1, 1, 1, 1, 4, 4, 4, 4,
#                     10, 10, 10, 10])
#    BATCH_SIZE = 12*N_GPUS
#elif int(sys.argv[1]) == 10000:
W = np.asarray([11, 11, 11, 11, 11, 11, 11, 11,
                21, 21, 21, 21, 41, 41, 41, 41])
AR = np.asarray([1, 1, 1, 1, 4, 4, 4, 4,
                 10, 10, 10, 10, 25, 25, 25, 25])
BATCH_SIZE = 6*N_GPUS
# Hyper-parameters:
# L: Number of convolution kernels
# W: Convolution window size in each residual unit
# AR: Atrous rate in each residual unit

CL = 2 * np.sum(AR*(W-1))

In [4]:
CL_max=10000
# Maximum nucleotide context length (CL_max/2 on either side of the 
# position of interest)
# CL_max should be an even number

SL=5000
# Sequence length of SpliceAIs (SL+CL will be the input length and
# SL will be the output length)

splice_table='/odinn/tmp/benediktj/SpliceAITrainingCode/canonical_dataset.txt'
ref_genome='/odinn/tmp/benediktj/SpliceAITrainingCode/hg19.fa'
# Input details

data_dir='/odinn/tmp/benediktj/SpliceAITrainingCode/'
sequence='/odinn/tmp/benediktj/SpliceAITrainingCode/canonical_sequence.txt'


In [5]:
assert CL_max % 2 == 0

IN_MAP = np.asarray([[0, 0, 0, 0],
                     [1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])
# One-hot encoding of the inputs: 0 is for padding, and 1, 2, 3, 4 correspond
# to A, C, G, T respectively.

OUT_MAP = np.asarray([[1, 0, 0],
                      [0, 1, 0],
                      [0, 0, 1],
                      [0, 0, 0]])
# One-hot encoding of the outputs: 0 is for no splice, 1 is for acceptor,
# 2 is for donor and -1 is for padding.


def ceil_div(x, y):
    return int(ceil(float(x)/y))


def create_datapoints(seq, strand, tx_start, tx_end, jn_start, jn_end):
    # This function first converts the sequence into an integer array, where
    # A, C, G, T, N are mapped to 1, 2, 3, 4, 0 respectively. If the strand is
    # negative, then reverse complementing is done. The splice junctions 
    # are also converted into an array of integers, where 0, 1, 2, -1 
    # correspond to no splicing, acceptor, donor and missing information
    # respectively. It then calls reformat_data and one_hot_encode
    # and returns X, Y which can be used by Keras models.

    seq = 'N'*(CL_max//2) + seq[CL_max//2:-CL_max//2] + 'N'*(CL_max//2)
    # Context being provided on the RNA and not the DNA

    seq = seq.upper().replace('A', '1').replace('C', '2')
    seq = seq.replace('G', '3').replace('T', '4').replace('N', '0')

    tx_start = int(tx_start)
    tx_end = int(tx_end) 

    jn_start = map(lambda x: map(int, re.split(',', x)[:-1]), jn_start)
    jn_end = map(lambda x: map(int, re.split(',', x)[:-1]), jn_end)

    if strand == '+':

        X0 = np.asarray(map(int, list(seq)))
        Y0 = [-np.ones(tx_end-tx_start+1) for t in range(1)]

        for t in range(1):
            
            if len(jn_start[t]) > 0:
                Y0[t] = np.zeros(tx_end-tx_start+1)
                for c in jn_start[t]:
                    if tx_start <= c <= tx_end:
                        Y0[t][c-tx_start] = 2
                for c in jn_end[t]:
                    if tx_start <= c <= tx_end:
                        Y0[t][c-tx_start] = 1
                    # Ignoring junctions outside annotated tx start/end sites
                     
    elif strand == '-':

        X0 = (5-np.asarray(map(int, list(seq[::-1])))) % 5  # Reverse complement
        Y0 = [-np.ones(tx_end-tx_start+1) for t in range(1)]

        for t in range(1):

            if len(jn_start[t]) > 0:
                Y0[t] = np.zeros(tx_end-tx_start+1)
                for c in jn_end[t]:
                    if tx_start <= c <= tx_end:
                        Y0[t][tx_end-c] = 2
                for c in jn_start[t]:
                    if tx_start <= c <= tx_end:
                        Y0[t][tx_end-c] = 1

    Xd, Yd = reformat_data(X0, Y0)
    X, Y = one_hot_encode(Xd, Yd)

    return X, Y


def reformat_data(X0, Y0):
    # This function converts X0, Y0 of the create_datapoints function into
    # blocks such that the data is broken down into data points where the
    # input is a sequence of length SL+CL_max corresponding to SL nucleotides
    # of interest and CL_max context nucleotides, the output is a sequence of
    # length SL corresponding to the splicing information of the nucleotides
    # of interest. The CL_max context nucleotides are such that they are
    # CL_max/2 on either side of the SL nucleotides of interest.

    num_points = ceil_div(len(Y0[0]), SL)

    Xd = np.zeros((num_points, SL+CL_max))
    Yd = [-np.ones((num_points, SL)) for t in range(1)]

    X0 = np.pad(X0, [0, SL], 'constant', constant_values=0)
    Y0 = [np.pad(Y0[t], [0, SL], 'constant', constant_values=-1)
         for t in range(1)]

    for i in range(num_points):
        Xd[i] = X0[SL*i:CL_max+SL*(i+1)]

    for t in range(1):
        for i in range(num_points):
            Yd[t][i] = Y0[t][SL*i:SL*(i+1)]

    return Xd, Yd


def clip_datapoints(X, Y, CL, N_GPUS):
    # This function is necessary to make sure of the following:
    # (i) Each time model_m.fit is called, the number of datapoints is a
    # multiple of N_GPUS. Failure to ensure this often results in crashes.
    # (ii) If the required context length is less than CL_max, then
    # appropriate clipping is done below.
    # Additionally, Y is also converted to a list (the .h5 files store 
    # them as an array).

    rem = X.shape[0]%N_GPUS
    clip = (CL_max-CL)//2

    if rem != 0 and clip != 0:
        return X[:-rem, clip:-clip], [Y[t][:-rem] for t in range(1)]
    elif rem == 0 and clip != 0:
        return X[:, clip:-clip], [Y[t] for t in range(1)]
    elif rem != 0 and clip == 0:
        return X[:-rem], [Y[t][:-rem] for t in range(1)]
    else:
        return X, [Y[t] for t in range(1)]


def one_hot_encode(Xd, Yd):

    return IN_MAP[Xd.astype('int8')], \
           [OUT_MAP[Yd[t].astype('int8')] for t in range(1)]



#def print_topl_statistics(y_true, y_pred):
#    # Prints the following information: top-kL statistics for k=0.5,1,2,4,
#    # auprc, thresholds for k=0.5,1,2,4, number of true splice sites.#

#    idx_true = np.nonzero(y_true == 1)[0]
#    argsorted_y_pred = np.argsort(y_pred)
#    sorted_y_pred = np.sort(y_pred)

#    topkl_accuracy = []
#    threshold = []

#    for top_length in [0.5, 1, 2, 4]:
#
#        idx_pred = argsorted_y_pred[-int(top_length*len(idx_true)):]

#        topkl_accuracy += [np.size(np.intersect1d(idx_true, idx_pred)) \
#                  / float(min(len(idx_pred), len(idx_true)))]
#        threshold += [sorted_y_pred[-int(top_length*len(idx_true))]]

#    auprc = average_precision_score(y_true, y_pred)

 #   print("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(
 #         np.round(topkl_accuracy[0],4), np.round(topkl_accuracy[1],4), np.round(topkl_accuracy[2],4),
#          np.round(topkl_accuracy[3],4), np.round(auprc,4), np.round(threshold[0],4), np.round(threshold[1],4),
 #         np.round(threshold[2],4), np.round(threshold[3],4), len(idx_true)))


In [6]:
def ResidualUnit(l, w, ar):
    # Residual unit proposed in "Identity mappings in Deep Residual Networks"
    # by He et al.

    def f(input_node):

        bn1 = BatchNormalization()(input_node)
        act1 = Activation('relu')(bn1)
        conv1 = Conv1D(l, [w], dilation_rate=[ar], padding='same')(act1)
        bn2 = BatchNormalization()(conv1)
        act2 = Activation('relu')(bn2)
        conv2 = Conv1D(l, [w], dilation_rate=[ar], padding='same')(act2)
        output_node = add([conv2, input_node])

        return output_node

    return f


def SpliceAI(L, W, AR):
    # L: Number of convolution kernels
    # W: Convolution window size in each residual unit
    # AR: Atrous rate in each residual unit

    assert len(W) == len(AR)

    CL = 2 * np.sum(AR*(W-1))

    input0 = Input(shape=(None, 4))
    conv = Conv1D(L, 1)(input0)
    skip = Conv1D(L, 1)(conv)

    for i in range(len(W)):
        conv = ResidualUnit(L, W[i], AR[i])(conv)
        
        if (((i+1) % 4 == 0) or ((i+1) == len(W))):
            # Skip connections to the output after every 4 residual units
            dense = Conv1D(L, 1)(conv)
            skip = add([skip, dense])

    skip = Cropping1D(int(CL/2))(skip)

    output0 = [[] for t in range(1)]

    for t in range(1):
        output0[t] = Conv1D(3, 1, activation='softmax')(skip)
    
    model = Model(inputs=input0, outputs=output0)

    return model


def categorical_crossentropy_2d(y_true, y_pred):
    # Standard categorical cross entropy for sequence outputs
    weights = [1, 1, 1]
    return - kb.mean(weights[0]*y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + weights[1]*y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10)
                   + weights[2]*y_true[:, :, 2]*kb.log(y_pred[:, :, 2]+1e-10))


In [7]:
def make_parallel(model, gpu_count):

    def get_slice(data, idx, parts):

        shape = tf.shape(data)
        stride = tf.concat([shape[:1]//parts, shape[1:]*0], 0)
        start = stride * idx

        size = tf.concat([shape[:1]//parts, shape[1:]], 0) 
        # Split the batch into equal parts 

        return tf.slice(data, start, size)

    outputs_all = []
    for i in range(len(model.outputs)):
        outputs_all.append([])

    # Place a copy of the model on each GPU, each getting a slice of the batch
    for i in range(gpu_count):
        with tf.device('/gpu:%d' % i):
            with tf.name_scope('tower_%d' % i) as scope:

                inputs = []
                # Slice each input into a piece for processing on this GPU
                for x in model.inputs:
                    input_shape = tuple(x.get_shape().as_list())[1:]
                    slice_n = Lambda(get_slice, output_shape=input_shape,
                                  arguments={'idx': i, 'parts': gpu_count})(x)
                    inputs.append(slice_n)

                outputs = model(inputs)
                
                if not isinstance(outputs, list):
                    outputs = [outputs]
                
                # Save all the outputs for merging back together later
                for l in range(len(outputs)):
                    outputs_all[l].append(outputs[l])

    # Merge outputs on CPU
    with tf.device('/cpu:0'):
        
        merged = []
        for outputs in outputs_all:
            merged.append(concatenate(outputs, axis=0))
            
        return Model(inputs=model.inputs, outputs=merged)

In [8]:


###############################################################################
# Training and validation
###############################################################################

h5f = h5py.File(data_dir + 'dataset' + '_' + 'train'
                + '_' + 'all_10k' + '.h5', 'r')



In [9]:
num_idx = len(h5f.keys())//2
idx_all = np.random.permutation(num_idx)
idx_train = idx_all[:int(0.9*num_idx)]
idx_valid = idx_all[int(0.9*num_idx):]

EPOCH_NUM = 10*len(idx_train)

n_null = 0
n_acceptor = 0
n_donor = 0
for idx in idx_train:
    y = h5f['Y' + str(idx)][:]
    n_null += np.sum(y[0,:,:,0]==1)
    n_acceptor += np.sum(y[0,:,:,1]==1)
    n_donor += np.sum(y[0,:,:,2]==1)

In [10]:
#np.save('../Data/idx_all.npy',idx_all)

In [11]:
#weights = (n_null+n_acceptor+n_donor) / (3 * np.array([n_null,n_acceptor,n_donor]))

In [12]:
weights = [1,1,1]

In [13]:
start_time = time.time()


In [14]:
for model_nr in range(5):
    model = SpliceAI(L, W, AR)
    model.save('../Results/TF_Models/SpliceAI_GENCODE_tf1_{}.h5'.format(model_nr))
    #model.summary()
    model_m = make_parallel(model, N_GPUS)
    model_m.compile(loss=categorical_crossentropy_2d, optimizer='adam')

    for epoch_num in tqdm(range(EPOCH_NUM)):

        idx = np.random.choice(idx_train)

        X = h5f['X' + str(idx)][:].astype(np.float32)
        Y = h5f['Y' + str(idx)][:].astype(np.float32)

        Xc, Yc = clip_datapoints(X, Y, CL, N_GPUS) 
        model_m.fit(Xc, Yc, batch_size=BATCH_SIZE, verbose=0)

        #if True:
        if (epoch_num+1) % len(idx_train) == 0:
            # Printing metrics (see utils.py for details)

            print("--------------------------------------------------------------")
            print ("\n\033[1mValidation set metrics:\033[0m")

            Y_true_1 = [[] for t in range(1)]
            Y_true_2 = [[] for t in range(1)]
            Y_pred_1 = [[] for t in range(1)]
            Y_pred_2 = [[] for t in range(1)]

            for idx in idx_valid:

                X = h5f['X' + str(idx)][:]
                Y = h5f['Y' + str(idx)][:]

                Xc, Yc = clip_datapoints(X, Y, CL, N_GPUS)
                Yp = model_m.predict(Xc, batch_size=BATCH_SIZE)

                if not isinstance(Yp, list):
                    Yp = [Yp]

                for t in range(1):

                    is_expr = (Yc[t].sum(axis=(1,2)) >= 1)

                    Y_true_1[t].extend(Yc[t][is_expr, :, 1].flatten())
                    Y_true_2[t].extend(Yc[t][is_expr, :, 2].flatten())
                    Y_pred_1[t].extend(Yp[t][is_expr, :, 1].flatten())
                    Y_pred_2[t].extend(Yp[t][is_expr, :, 2].flatten())

            print("\n\033[1mAcceptor:\033[0m")
            for t in range(1):
                print_topl_statistics(np.asarray(Y_true_1[t]),
                                      np.asarray(Y_pred_1[t]))

            print("\n\033[1mDonor:\033[0m")
            for t in range(1):
                print_topl_statistics(np.asarray(Y_true_2[t]),
                                      np.asarray(Y_pred_2[t]))

            print("\n\033[1mTraining set metrics:\033[0m")

            Y_true_1 = [[] for t in range(1)]
            Y_true_2 = [[] for t in range(1)]
            Y_pred_1 = [[] for t in range(1)]
            Y_pred_2 = [[] for t in range(1)]

            for idx in idx_train[:len(idx_valid)]:

                X = h5f['X' + str(idx)][:]
                Y = h5f['Y' + str(idx)][:]

                Xc, Yc = clip_datapoints(X, Y, CL, N_GPUS)
                Yp = model_m.predict(Xc, batch_size=BATCH_SIZE)

                if not isinstance(Yp, list):
                    Yp = [Yp]

                for t in range(1):

                    is_expr = (Yc[t].sum(axis=(1,2)) >= 1)

                    Y_true_1[t].extend(Yc[t][is_expr, :, 1].flatten())
                    Y_true_2[t].extend(Yc[t][is_expr, :, 2].flatten())
                    Y_pred_1[t].extend(Yp[t][is_expr, :, 1].flatten())
                    Y_pred_2[t].extend(Yp[t][is_expr, :, 2].flatten())

            print("\n\033[1mAcceptor:\033[0m")
            for t in range(1):
                print_topl_statistics(np.asarray(Y_true_1[t]),
                                      np.asarray(Y_pred_1[t]))

            print("\n\033[1mDonor:\033[0m")
            for t in range(1):
                print_topl_statistics(np.asarray(Y_true_2[t]),
                                      np.asarray(Y_pred_2[t]))

            print("Learning rate: {}".format(kb.get_value(model_m.optimizer.lr)))
            print("--- {} seconds ---".format((time.time() - start_time)))
            start_time = time.time()

            print("--------------------------------------------------------------")

            model.save('../Results/TF_Models/SpliceAI_GENCODE_tf1_trained_{}_c.h5'.format(model_nr))

            if (epoch_num+1) >= 6*len(idx_train):
                kb.set_value(model_m.optimizer.lr,
                             0.5*kb.get_value(model_m.optimizer.lr))
                # Learning rate decay



2022-07-07 17:04:17.662272: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3000000000 Hz
2022-07-07 17:04:17.665183: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x6d870c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-07-07 17:04:17.665220: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-07-07 17:04:17.667159: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-07 17:04:19.587621: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x6de66c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-07-07 17:04:19.587655: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA Tesla V100-PCIE-32GB, Compute Capability 7.0
2022-07-07 17:04:19.587660: I tensorflow/compiler/xla/service/service.cc:176]   

2022-07-07 17:04:20.187689: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1206] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-07-07 17:04:20.187737: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1212]      0 1 
2022-07-07 17:04:20.187748: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1225] 0:   N Y 
2022-07-07 17:04:20.187753: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1225] 1:   Y N 
2022-07-07 17:04:20.190585: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1351] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 31013 MB memory) -> physical GPU (device: 0, name: NVIDIA Tesla V100-PCIE-32GB, pci bus id: 0000:5e:00.0, compute capability: 7.0)
2022-07-07 17:04:20.191988: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1351] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:1 with 31013 MB memory) -> physical GPU (device: 1, name: NVIDIA Tesla V100-PCIE-32GB, pci bus id: 0000:86:00.0, compu

  0%|                                                                                                                          | 0/1190 [00:00<?, ?it/s]

2022-07-07 17:05:21.259850: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-07-07 17:05:22.622654: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
 10%|██████████▉                                                                                                   | 118/1190 [34:09<2:37:41,  8.83s/it]

--------------------------------------------------------------

Validation set metrics:

Acceptor:
0.9822	0.8455	0.9636	0.9841	0.915	0.9766	0.6623	0.0686	0.0083	13051	15435.0	15435

Donor:
0.9815	0.8532	0.9707	0.9888	0.9213	0.9936	0.8825	0.2132	0.0296	13173	15440.0	15440

Training set metrics:

Acceptor:
0.982	0.8649	0.9708	0.9856	0.9271	0.9793	0.6241	0.0432	0.0053	11313	13080.0	13080

Donor:
0.9839	0.8766	0.9797	0.9905	0.9387	0.9937	0.8546	0.1301	0.0163	11469	13084.0	13084
Learning rate: 0.0010000000474974513
--- 2724.6832313537598 seconds ---
--------------------------------------------------------------


...

 90%|██████████████████████████████████████████████████████████████████████████████████████████████████           | 1070/1190 [4:15:36<13:51,  6.93s/it]

--------------------------------------------------------------

Validation set metrics:

Acceptor:
0.9908	0.9308	0.9867	0.9908	0.9675	0.9890	0.4907	0.0077	0.0021	14367	15435.0	15435

Donor:
0.9902	0.9363	0.988	0.9919	0.9701	0.9910	0.5164	0.0066	0.0019	14456	15440.0	15440

Training set metrics:

Acceptor:
0.9985	0.9673	0.995	0.9965	0.9895	0.9896	0.4962	0.0064	0.0020	12652	13080.0	13080

Donor:
0.9983	0.9715	0.9956	0.9969	0.991	0.9917	0.5102	0.0053	0.0017	12711	13084.0	13084
Learning rate: 0.0001250000059371814
--- 1736.43976354599 seconds ---
--------------------------------------------------------------


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 1189/1190 [4:44:01<00:10, 10.70s/it]

--------------------------------------------------------------

Validation set metrics:

Acceptor:
0.9917	0.9341	0.9863	0.9912	0.9694	0.9926	0.3832	0.0026	0.0006	14418	15435.0	15435

Donor:
0.9904	0.9379	0.9874	0.9923	0.9707	0.9935	0.4407	0.0026	0.0006	14481	15440.0	15440

Training set metrics:

Acceptor:
0.9989	0.9716	0.9955	0.9971	0.9913	0.9922	0.4005	0.0022	0.0006	12708	13080.0	13080

Donor:
0.9986	0.975	0.9965	0.9972	0.9924	0.9935	0.4501	0.0019	0.0005	12757	13084.0	13084
Learning rate: 6.25000029685907e-05
--- 1696.2944524288177 seconds ---
--------------------------------------------------------------


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1190/1190 [4:51:05<00:00, 14.68s/it]


In [15]:
h5f.close()

In [8]:
from tensorflow.keras.models import load_model 

In [9]:
h5f = h5py.File(data_dir + 'dataset' + '_' + 'test'
                + '_' + '0_10k' + '.h5', 'r')

num_idx = len(h5f.keys())//2

#model = load_model('../Results/TF_Models/SpliceAI_{}_c.h5'.format(0))
model = []
n_models = 5

for v in range(n_models):
    model.append(load_model('../Results/TF_Models/SpliceAI_GENCODE_tf1_trained_{}_c.h5'.format(v)))


###############################################################################
# Model testing
###############################################################################

start_time = time.time()

output_class_labels = ['Null', 'Acceptor', 'Donor']
# The three neurons per output correspond to no splicing, splice acceptor (AG)
# and splice donor (GT) respectively.

#chunkSize = num_idx/10
#for i in range(10):
Y_true_acceptor = []
Y_pred_acceptor = []
Y_true_donor = []
Y_pred_donor = []
ce_2d = []

for idx in tqdm(range(num_idx)):
    X = h5f['X' + str(idx)][:]
    Y = h5f['Y' + str(idx)][:]

    Xc, Yc = clip_datapoints(X, Y, CL, 1)

    Yps = [np.zeros(Yc[0].shape) for t in range(1)]

    for v in range(n_models):
        Yp = model[v].predict(Xc, batch_size=BATCH_SIZE)

        if not isinstance(Yp, list):
            Yp = [Yp]


        Yps[0] += Yp[0]/n_models
    # Ensemble averaging (mean of the ensemble predictions is used)

    ce_2d.append(cross_entropy_2d(Yc[0],Yps[0]))
    is_expr = (Yc[0].sum(axis=(1,2)) >= 1)

    Y_true_acceptor.extend(Yc[0][is_expr, :, 1].flatten())
    Y_pred_acceptor.extend(Yps[0][is_expr, :, 1].flatten())
    Y_true_donor.extend(Yc[0][is_expr, :, 2].flatten())
    Y_pred_donor.extend(Yps[0][is_expr, :, 2].flatten())



Y_true_acceptor = np.asarray(Y_true_acceptor)
Y_pred_acceptor = np.asarray(Y_pred_acceptor)
Y_true_donor = np.asarray(Y_true_donor)
Y_pred_donor = np.asarray(Y_pred_donor)

mean_ce = np.mean(ce_2d)
print('Cross entropy = {}'.format(mean_ce))

print("\n\033[1m{}:\033[0m".format(output_class_labels[1]))

acceptor_results = print_topl_statistics(Y_true_acceptor, Y_pred_acceptor)

print("\n\033[1m{}:\033[0m".format(output_class_labels[2]))

donor_results = print_topl_statistics(Y_true_donor, Y_pred_donor)

h5f.close()

print("--- {} seconds ---".format(time.time() - start_time))
print("--------------------------------------------------------------")

2022-07-11 10:45:44.173486: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-11 10:45:45.917655: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: NVIDIA Tesla V100-PCIE-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:5e:00.0
2022-07-11 10:45:45.918550: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 1 with properties: 
name: NVIDIA Tesla V100-PCIE-32GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:86:00.0
2022-07-11 10:45:45.918584: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-11 10:45:46.052411: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-07-11 10:45:46.146318: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library l

  0%|                                                                                                | 0/16 [00:00<?, ?it/s]2022-07-11 10:46:18.679420: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-07-11 10:46:25.121761: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
100%|███████████████████████████████████████████████████████████████████████████████████████| 16/16 [06:04<00:00, 22.79s/it]


Cross entropy = 0.0002020825685298107

Acceptor:
0.9948	0.9409	0.9852	0.9903	0.9738	0.9857	0.2938	0.0036	0.0007	13445	14289.0	14289

Donor:
0.9954	0.9453	0.9889	0.9929	0.977	0.9866	0.3222	0.0030	0.0006	13508	14289.0	14289
--- 503.026394367218 seconds ---
--------------------------------------------------------------


In [14]:
data_dir = '../Data'
setType = 'test'
annotation_test, transcriptToLabel_test, seqData = getData(data_dir, setType)

In [18]:
temp = 1
n_models = 5
device = torch.device("cpu")

output_class_labels = ['Null', 'Acceptor', 'Donor']

model = []
n_models = 5

for v in range(n_models):
    model.append(load_model('../Results/TF_Models/SpliceAI_GENCODE_tf1_trained_{}_c.h5'.format(v)))







2022-10-10 11:26:26.309839: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-10-10 11:26:28.371284: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: NVIDIA A100-PCIE-40GB major: 8 minor: 0 memoryClockRate(GHz): 1.41
pciBusID: 0000:31:00.0
2022-10-10 11:26:28.371476: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 1 with properties: 
name: NVIDIA A100-PCIE-40GB major: 8 minor: 0 memoryClockRate(GHz): 1.41
pciBusID: 0000:98:00.0
2022-10-10 11:26:28.371627: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 2 with properties: 
name: NVIDIA A100-PCIE-40GB major: 8 minor: 0 memoryClockRate(GHz): 1.41
pciBusID: 0000:ca:00.0
2022-10-10 11:26:28.371647: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-10-10 11:26:31.455728: I tensorflow/stream_executor/platform/defau

In [55]:
Y_true_acceptor, Y_pred_acceptor = [],[]
Y_true_donor, Y_pred_donor = [],[]
ce_2d = []

test_dataset = spliceDataset(getDataPointListFull(annotation_test,transcriptToLabel_test,SL,CL_max,shift=SL))
test_dataset.seqData = seqData
BATCH_SIZE=100
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)


#targets_list = []
#outputs_list = []
ce_2d = []
for (batch_features ,targets) in tqdm(test_loader):
    Xc = torch.transpose(batch_features,1,2).numpy()
    targets = torch.transpose(targets[:,:,CL_max//2:-CL_max//2],1,2).numpy()
    outputs = []
    for v in range(n_models):
        Yp = model[v].predict(Xc, batch_size=BATCH_SIZE)
        outputs.append(Yp)
    outputs = (outputs[0]+outputs[1]+outputs[2]+outputs[3]+outputs[4])/n_models
    #outputs = np.swapaxes(outputs,1,2)
    ce_2d.append(cross_entropy_2d(targets,outputs))

    is_expr = (targets.sum(axis=(1,2)) >= 1)
    Y_true_acceptor.extend(targets[is_expr, :, 1].flatten())
    Y_true_donor.extend(targets[is_expr, :, 2].flatten())
    Y_pred_acceptor.extend(outputs[is_expr, :, 1].flatten())
    Y_pred_donor.extend(outputs[is_expr, :, 2].flatten())

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1331/1331 [23:50<00:00,  1.07s/it]


In [56]:
mean_ce = np.mean(ce_2d)
print('Cross entropy = {}'.format(mean_ce))
Y_true_acceptor, Y_pred_acceptor,Y_true_donor, Y_pred_donor = np.array(Y_true_acceptor), np.array(Y_pred_acceptor),np.array(Y_true_donor), np.array(Y_pred_donor)
print("\n\033[1m{}:\033[0m".format('Acceptor'))
acceptor_val_results = print_topl_statistics(Y_true_acceptor, Y_pred_acceptor)
print("\n\033[1m{}:\033[0m".format('Donor'))
donor_val_results =print_topl_statistics(Y_true_donor, Y_pred_donor)

Cross entropy = 0.00012475274132396565

Acceptor:
0.9796	0.9325	0.9882	0.9932	0.9602	0.9881	0.4308	0.0035	0.0006	83655	89712.0	89712

Donor:
0.9802	0.9378	0.9916	0.9952	0.965	0.9888	0.4790	0.0029	0.0005	84128	89712.0	89712
